<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
''' Imports and such  '''
!pip install keras
import pandas as pd
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras import backend
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [2]:
''' Brining in the data '''
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = df.drop(columns={'customerID'})
print(df.shape)
df.head()

(7043, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
cleaning_up = {
    'gender' : {'male' : 1, 'female' :  0, 'Male' : 1, 'Female' : 0},
    'Partner' : {'Yes' : 1, 'No' : 0},
    'Dependents' : {'Yes' : 1, 'No' : 0},
    'PhoneService': {'Yes' : 1, 'No' : 0},
    'MultipleLines' : {'Yes' : 1, 'No' : 0, 'No phone service' : -1},
    'InternetService' : {'Fiber optic' : 0, 'DSL' : 1, 'No' : -1, 'No internet service' : -1, "'No internet service'" : -1},
    'OnlineSecurity' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'OnlineBackup' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'DeviceProtection' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'TechSupport' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'StreamingTV' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'StreamingMovies' : {'Yes' : 1, 'No' : 0, 'No internet service' : -1},
    'Contract' : {'Month-to-month': 0, 'One year' : 1, 'Two year' : 2},
    'PaperlessBilling' : {'Yes' : 1, 'No' : 0},
    'PaymentMethod' : {'Electronic check' : 0, 'Mailed check' : 1, 'Bank transfer (automatic)' : 2, 'Credit card (automatic)' : 3},
    'Churn' : {'Yes' : 1, 'No' : 0},
    #'TotalCharges' : {' ' : ''}
    
}

In [4]:
df.replace(cleaning_up, inplace=True)
#df = df.drop(columns='TotalCharges')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,-1,1,0,1,0,0,0,0,0,1,0,29.85,29.85,0
1,1,0,0,0,34,1,0,1,1,0,1,0,0,0,1,0,1,56.95,1889.5,0
2,1,0,0,0,2,1,0,1,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,1,0,0,0,45,0,-1,1,1,0,1,1,0,0,1,0,2,42.30,1840.75,0
4,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,70.70,151.65,1


In [5]:
df['InternetService'] = df['InternetService'].astype(int)
df['TotalCharges'] = df['TotalCharges'].str.strip()
#df['MonthlyCharges'] = int(df['MonthlyCharges'])

In [6]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [7]:
df['TotalCharges'].value_counts()

           11
20.2       11
19.75       9
19.9        8
20.05       8
19.65       8
19.55       7
45.3        7
19.45       6
20.15       6
20.25       6
20.45       5
20.3        5
19.85       4
19.95       4
69.95       4
69.65       4
69.6        4
69.9        4
74.7        4
44.4        4
75.3        4
19.5        4
50.15       4
44          4
19.4        4
20.35       4
20.4        4
20.5        4
19.2        4
           ..
6671.7      1
856.35      1
1530.6      1
6038.55     1
1887        1
408.5       1
2447.95     1
181.1       1
92.45       1
5555.3      1
5430.65     1
1761.45     1
1226.45     1
469.8       1
834.1       1
1152.7      1
2680.15     1
6109.75     1
1442.6      1
7843.55     1
5688.45     1
225.75      1
237.7       1
48.75       1
6518.35     1
6236.75     1
3856.75     1
6314.35     1
84.75       1
2088.75     1
Name: TotalCharges, Length: 6531, dtype: int64

In [10]:
df['TotalCharges'] = df['TotalCharges'].str.contains(' ',  na=np.nan)
df = df.dropna()


In [11]:
df.shape

(7043, 20)

In [12]:
''' Removing the Target Variables and Such '''
X = df.loc[:, df.columns != 'Churn']
y = df.loc[:, df.columns == 'Churn']

In [13]:
''' All the categoricals are nice and low in cardinality, 
so gonna one hot encode all of em 
'''

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

#X = encoder.fit_transform(X)


In [15]:
''' Creating a Sexy Model '''
''' The parameters to Tun '''
input_dim = (len(df.columns) - 1)
epochs = 10
batch_size=10
momentum= 0.01
#pt = SGD(lr=0.2)

''' Building the Model '''
def create_model():
    model = Sequential()

    model.add(Dense(input_dim*2, input_dim=input_dim, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(input_dim**2, activation='sigmoid'))
    
    model.add(Dense(1, activation='sigmoid'))


    model.compile(loss = 'mean_absolute_percentage_error',
                  optimizer=RMSprop(lr=.2),
                  metrics = ['accuracy'],)
                  #lr = lr,
                  #momentum = momentum)
    return model

''' Fitting the Model '''
''' 
#history = model.fit(X, y,
                    batch_size=batch_size,
                    epochs = epochs,
                    verbose=1,
                   #validation_split=.1
                   ) 
                   '''

''' Grid Search CV '''
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10],
              'epochs': [5],
              #'momentum' : [.01, .1, .2, .3, ],
             # 'lr' : [.01]
             }

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7346301292105356 using {'batch_size': 10, 'epochs': 5}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?